In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [184]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df=pd.concat([df_train.drop(['target'],axis=1),df_test],axis=0,sort=False)

In [3]:
df.dropna(inplace = True)

In [4]:
df.drop(labels ='id', axis=1 , inplace= True)

In [5]:
df.to_csv('df_ok.csv', index = False)

## here i use the R package 'arules' to generate all association rules

In [32]:
dfRulesL2 = pd.read_csv('df_rules_L_2.csv', index_col= 0)
dfRulesL2.drop(labels =['lift', 'count'], axis=1 , inplace= True)
dfRulesL2.head()

,rules,support,confidence
1,nom_6=f19fdbb3b => bin_0=0,0.001004,0.923933
2,bin_0=0 => nom_6=f19fdbb3b,0.001004,0.001106
3,nom_5=a16f19e9b => bin_0=0,0.001008,0.925926
4,bin_0=0 => nom_5=a16f19e9b,0.001008,0.001111
5,nom_5=c0e9469c1 => bin_0=0,0.001006,0.920664


In [85]:
dfRulesL1 = pd.read_csv('df_rules_L_1.csv', index_col= 0)
dfRulesL1.head()

,rules,Support & confidence
1,nom_5=6293a27d6,0.001002
2,nom_5=1e78a3926,0.001002
3,nom_5=5b707c4fb,0.001002
4,nom_5=3e9044b40,0.001004
5,nom_5=5ea48fdbd,0.001004


In [139]:
gain = []
minconf  = 0.001
def add_gain(row):
    LHS = row[0].split('=>')[0].strip()
    #print(LHS)
    supLHS = float(dfRulesL1[dfRulesL1['rules'] == LHS].iloc[0,1])
    gain.append([(row[2]-minconf)* supLHS])

dfRulesL2.apply(add_gain,axis= 1)


1        None
2        None
3        None
4        None
5        None
         ... 
41846    None
41847    None
41848    None
41849    None
41850    None
Length: 41850, dtype: object

In [140]:
AV = []
def add_added_value(row):
    RHS = row[0].split('=>')[1].strip()
    #print(LHS)
    supRHS = float(dfRulesL1[dfRulesL1['rules'] == RHS].iloc[0,1])
    AV.append([row[2] - supRHS])

dfRulesL2.apply(add_added_value,axis= 1)

1        None
2        None
3        None
4        None
5        None
         ... 
41846    None
41847    None
41848    None
41849    None
41850    None
Length: 41850, dtype: object

In [144]:
dfRulesL2['gain'] = gain
dfRulesL2['added value']  = AV

In [154]:
def unlist(row):
    row[3]  = row[3][0]
    row[4]  = row[4][0]
    return row
dfRulesL2 = dfRulesL2.apply(unlist, axis = 1)

In [156]:
dfRulesL2Filtered = dfRulesL2[dfRulesL2['gain'] > 0 ]
dfRulesL2Filtered = dfRulesL2[dfRulesL2['added value'] > 0] 
dfRulesL2Filtered =  dfRulesL2[dfRulesL2['added value'] < 0]
dfRulesL2Filtered.head()

In [176]:
#creat an Filtered-ARN
f_arn = nx.DiGraph()


In [177]:
dfRulesL2Filtered.apply(lambda row: f_arn.add_node(row[0]),axis= 1)

33       None
34       None
57       None
58       None
71       None
         ... 
41844    None
41847    None
41848    None
41849    None
41850    None
Length: 20396, dtype: object

In [178]:
dfRulesL2Filtered.apply(lambda row: f_arn.add_edge(row[0].split('=>')[0].strip(),row[0].split('=>')[1].strip() ),axis= 1)

33       None
34       None
57       None
58       None
71       None
         ... 
41844    None
41847    None
41848    None
41849    None
41850    None
Length: 20396, dtype: object

In [180]:
f_arn.

networkx.classes.digraph.DiGraph

In [186]:
df.iloc[3,:]

id               3
bin_0          NaN
bin_1            0
bin_2            0
bin_3            F
bin_4            N
nom_0          Red
nom_1       Circle
nom_2      Hamster
nom_3      Finland
nom_4     Theremin
nom_5    673bdf1f6
nom_6    23edb8da3
nom_7    3a33ef960
nom_8    bdaa56dd1
nom_9    f9d456e57
ord_0            1
ord_1       Novice
ord_2     Lava Hot
ord_3            a
ord_4            C
ord_5          NaN
day              3
month            3
Name: 3, dtype: object